# Data Modeling with PostgreSQL

In [ ]:
%load_ext autoreload
%autoreload 2

import pathlib
import psycopg2
import pandas as pd

from create_tables import init_database_and_tables
from sql_queries import (
    song_table_insert, song_table_check,
    artist_table_insert, artist_table_check,
    time_table_insert, time_table_check,
    user_table_insert, user_tabel_check
)

In [ ]:
init_database_and_tables()

In [ ]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb port=5432 user=student password=student")

print("PostgreSQL server information")
print(conn.get_dsn_parameters(), "\n")

cur = conn.cursor()

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [ ]:
song_files = list(pathlib.Path("data/song_data").rglob("*.json"))

In [ ]:
df = pd.read_json(song_files[0], typ='series')
df

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [ ]:
song_data = df[["song_id", "title", "artist_id", "year", "duration"]]
song_data

In [ ]:
cur.execute(song_table_insert, song_data.values)
conn.commit()

In [ ]:
cur.execute(song_table_check)

for row in cur.fetchall():
    print(row)

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [ ]:
artist_data = df[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]]
artist_data

In [ ]:
cur.execute(artist_table_insert, artist_data.values)
conn.commit()

In [ ]:
cur.execute(artist_table_check)

for row in cur.fetchall():
    print(row)

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [ ]:
log_files = list(pathlib.Path("data/log_data").rglob("*.json"))

In [ ]:
df = pd.read_json(log_files[0], lines=True)
df.head()

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetime-like properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
df = df[df['page'] == 'NextSong']
df.head()

In [ ]:
t = pd.to_datetime(df['ts'], unit='ms')
t.head()

In [ ]:
dt = t.dt
time_data = (t, dt.hour, dt.day, dt.isocalendar().week, dt.month, dt.year, dt.weekday)
column_labels = ("timestamp", "hour", "day", "week", "month", "year", "weekday")

In [ ]:
time_df = pd.DataFrame({k: v for k, v in zip(column_labels, time_data)})
time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for _, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

In [ ]:
cur.execute(time_table_check)

for row in cur.fetchall():
    print(row)

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df = user_df.drop_duplicates('userId')
user_df.head()

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

In [ ]:
cur.execute(user_tabel_check)

for row in cur.fetchall():
    print(row)

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()